In [2]:
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt

In [10]:
#originally downloaded here: https://gis-kingcounty.opendata.arcgis.com/datasets/transit-stops-for-king-county-metro-transitstop-point/explore
gdf_stops = gpd.read_file("data/transit/metro/Transit_Stops_for_King_County_Metro___transitstop_point.zip")
#made by the metro_data_cleaning.ipynb notebook in this repo
df_ridership = pd.read_parquet("data/cleaned_metro_ridership.parquet")

gdf_merged = gdf_stops.merge(
    df_ridership,
    left_on="STOP_ID",
    right_on="stop_id",
)

In [34]:
gdf_less = gdf_merged[[
    "geometry",
    "route_number",
    "stop_id",
    "postpandemic_avg_daily_boardings",
    "prepandemic_avg_daily_boardings",
]].copy()
gdf_less = gdf_less.to_crs("EPSG:3857") # convert to web mercator
gdf_less["geometry"] = gdf_less["geometry"].buffer(100)
gdf_less["post:pre_ridership_ratio"] = (gdf_less["postpandemic_avg_daily_boardings"].fillna(0) + 0.01) / (gdf_less["prepandemic_avg_daily_boardings"].fillna(0) + 0.01)
gdf_less["post:pre_ridership_ratio"] = gdf_less["post:pre_ridership_ratio"].apply(lambda r: r if r < 3 else 3)

In [38]:
gdf_less[gdf_less["route_number"] == 255].drop_duplicates(subset="stop_id").explore(
    column="post:pre_ridership_ratio"
)

In [37]:
gdf_less[gdf_less["route_number"] == 255].sort_values("postpandemic_avg_daily_boardings")

,geometry,route_number,stop_id,postpandemic_avg_daily_boardings,prepandemic_avg_daily_boardings,post:pre_ridership_ratio
533,"POLYGON ((-13604256.426 6056335.212, -13604256...",255,70510,0.1,0.0,3.000000
536,"POLYGON ((-13604256.426 6056335.212, -13604256...",255,70510,0.1,0.0,3.000000
2811,"POLYGON ((-13602715.767 6048868.559, -13602716...",255,74530,0.1,3.4,0.032258
530,"POLYGON ((-13604256.426 6056335.212, -13604256...",255,70510,0.1,0.0,3.000000
2812,"POLYGON ((-13602715.767 6048868.559, -13602716...",255,74530,0.1,3.4,0.032258
...,...,...,...,...,...,...
2605,"POLYGON ((-13617788.173 6041092.892, -13617788...",255,532,NaN,216.9,0.000046
2606,"POLYGON ((-13617788.173 6041092.892, -13617788...",255,532,NaN,216.9,0.000046
2607,"POLYGON ((-13618299.509 6041926.902, -13618299...",255,565,NaN,373.2,0.000027
2608,"POLYGON ((-13618299.509 6041926.902, -13618299...",255,565,NaN,373.2,0.000027
